In [26]:
# really importans stuff: this gets us time at utc 15 minutes ago
import pendulum
pendulum.now().set(second=0, microsecond=0).in_timezone("UTC").subtract(minutes=15)

DateTime(2025, 12, 16, 3, 24, 0, tzinfo=Timezone('UTC'))

In [14]:
import json
import requests

from pathlib import Path
from typing import Optional
from urllib.parse import urlencode

credentials = {
        "key": "PKCTYL4MO5SA2QEIZL2TDEJRTA",
        "secret_key": "DKL2eLVYFcxKzMJtDYbuG3zppWjpwHzsTS1DtHVwc9Cz",
        "finnhub": "d4ofku9r01quuso9dtkgd4ofku9r01quuso9dtl0"
    }
headers_alpaca = {
    "accept": "application/json",
    "APCA-API-KEY-ID": credentials["key"],
    "APCA-API-SECRET-KEY": credentials["secret_key"]
}


def get_tickers(path_to_json: Optional[Path] = None) -> list:
    if path_to_json is None:
        return ['aapl', 'nvda']
    with open(path_to_json, "r") as tickers_file:
        tickers = json.load(tickers_file)
    return tickers

    
def get_latest_bars(tickers_to_search, headers):
    
    params = {
        'symbols': ",".join(tickers_to_search),
        }
    url = f"https://data.alpaca.markets/v2/stocks/bars/latest?{urlencode(params)}"
    response = requests.get(url, headers=headers)
    return response.text

In [15]:
tickers = get_tickers()
latest_bars = get_latest_bars(tickers, headers_alpaca)
type(latest_bars)

dict

In [35]:
# no taskflow
import pendulum
import tasks_alpaca

from airflow.providers.standard.operators.python import PythonOperator
from airflow.sdk import DAG
from minio import Minio
from pathlib import Path

# minio conn
login_minio, password_minio = tasks_alpaca.get_minio_credentials()
host = "localhost"
client = Minio(
    endpoint=f"{host}:9000",
    access_key=login_minio,
    secret_key=password_minio,
    secure=False
    )


with DAG(
    dag_id="get_latest_bars",
    catchup=False,
    tags="stonks",
    start_date=pendulum.now().set(second=0, microsecond=0).in_timezone("UTC").subtract(minutes=15),
    schedule=pendulum.duration(minutes=1)
) as dag:
    
    tickers = PythonOperator(
        task_id="get_ticker_list",
        python_callable=tasks_alpaca.get_tickers,
        op_kwargs={"path_to_json": Path().cwd() / "data" / "tickers.json"},
        dag=dag)
    bars = PythonOperator(
        task_id="get_latest_bars",
        python_callable=tasks_alpaca.get_latest_bars,
        op_kwargs={"tickers_to_search": tickers},
        dag=dag)
    save_data = PythonOperator(
        task_id="save_data_to_bronze_bucket",
        python_callable=tasks_alpaca.upload_to_bucket,
        op_kwargs={
            "source_file":bars,
            "bucket_name":"learning.bronze",
            "client": client,
            "destination_file":dag.start_date
        })
    

2025-12-16 04:10:00+00:00


In [14]:
# realtime taskflow
import io
import json
import os
import pendulum 
import requests

from airflow.sdk import dag, task
from dotenv import load_dotenv
from minio import Minio
from minio.error import S3Error
from pathlib import Path
from tasks_alpaca import get_minio_credentials, upload_to_bucket
from typing import Union, Tuple, Optional
from urllib.parse import urlencode

credentials = {
        "key": "PKCTYL4MO5SA2QEIZL2TDEJRTA",
        "secret_key": "DKL2eLVYFcxKzMJtDYbuG3zppWjpwHzsTS1DtHVwc9Cz"
    }
headers_alpaca = {
    "accept": "application/json",
    "APCA-API-KEY-ID": credentials["key"],
    "APCA-API-SECRET-KEY": credentials["secret_key"]
}

# minio conn
login_minio, password_minio = get_minio_credentials()
host = "localhost"
client = Minio(
    endpoint=f"{host}:9000",
    access_key=login_minio,
    secret_key=password_minio,
    secure=False
    )


@dag(
    dag_id="get_latest_bars",
    catchup=False,
    tags="stonks",
    start_date=pendulum.now().set(second=0, microsecond=0).in_timezone("UTC").subtract(minutes=15),
    schedule=pendulum.duration(minutes=1))
def get_realtime_tickers():
    
    @task
    def get_tickers(path_to_json: Optional[Path] = None) -> list:
        if path_to_json is None:
            return {"tickers_to_search": ['aapl', 'nvda']}
        with open(path_to_json, "r") as tickers_file:
            tickers = json.load(tickers_file)
        return {"tickers_to_search": tickers}
        
    @task
    def get_latest_bars(tickers_to_search):
        print(tickers_to_search)
        params = {
            'symbols': ",".join(tickers_to_search),
            }
        url = f"https://data.alpaca.markets/v2/stocks/bars/latest?{urlencode(params)}"
        response = requests.get(url, headers=headers_alpaca)
        return response.text
        
    @task
    upload_to_bucket()

    tickers = get_tickers()
    bars = get_latest_bars(tickers)
    upload_to_bucket(source_file=bars, bucket_name="airflow.learn", client=client, destination_file="{{dag_run.start_date}}")

In [15]:
get_realtime_tickers()

<DAG: get_latest_bars>

In [44]:
# realtime taskflow
import io
import json
import os
import pendulum 
import requests

from airflow.sdk import dag, task
from dotenv import load_dotenv
from minio import Minio
from minio.error import S3Error
from pathlib import Path
from tasks_alpaca import get_minio_credentials, create_bucket
from typing import Union, Tuple, Optional
from urllib.parse import urlencode

credentials = {
        "key": "PKCTYL4MO5SA2QEIZL2TDEJRTA",
        "secret_key": "DKL2eLVYFcxKzMJtDYbuG3zppWjpwHzsTS1DtHVwc9Cz"
    }
headers_alpaca = {
    "accept": "application/json",
    "APCA-API-KEY-ID": credentials["key"],
    "APCA-API-SECRET-KEY": credentials["secret_key"]
}

# minio conn
login_minio, password_minio = get_minio_credentials()
host = "localhost"
client = Minio(
    endpoint=f"{host}:9001",
    access_key=login_minio,
    secret_key=password_minio,
    secure=False
    )

def get_realtime_tickers():
    @task
    def get_tickers(path_to_json: Optional[Path] = None) -> list:
        if path_to_json is None:
            return ['aapl', 'nvda']
        with open(path_to_json, "r") as tickers_file:
            tickers = json.load(tickers_file)
        return tickers
    @task
    def get_latest_bars(tickers_to_search):
        print(tickers_to_search)
        params = {
            'symbols': ",".join(tickers_to_search),
            }
        url = f"https://data.alpaca.markets/v2/stocks/bars/latest?{urlencode(params)}"
        response = requests.get(url, headers=headers_alpaca)
        return response.text
    @task    
    def upload_to_bucket(source_file: Union[str, Path, bytes], bucket_name: str, client: Minio, destination_file: str=None) -> None:
        # Make the bucket if it doesn't exist.
        create_bucket(bucket_name, client)
        # Encode str to bytes
        file_encoded = source_file.encode("utf-8")
        file_to_upload = io.BytesIO(file_encoded)
        # Upload file
        client.put_object(
            data=file_to_upload,
            bucket_name=bucket_name,
            object_name=destination_file,
            length=len(file_encoded)
        )
        print("Successfully uploaded object", destination_file, "to bucket", bucket_name)

    tickers = get_tickers()
    bars = get_latest_bars(tickers)
    upload_to_bucket(source_file=bars, bucket_name="airflow.learn", client=client, destination_file="{{dag_run.start_date}}")
    

In [45]:
get_realtime_tickers()

['aapl', 'nvda']
Bucket airflow.learn already exists
Successfully uploaded object test_upload.txt to bucket airflow.learn


In [46]:
Path("airflow_learning/data/tickers.json")

PosixPath('airflow_learning/data/tickers.json')

In [48]:
pwd

'/home/heinz/python-workspace/stonks/airflow_learning/dags'

In [50]:
with open (Path("/home/heinz/python-workspace/stonks/data/latest"), "rb") as file:
    bars = file.read().decode()
bars

'asss'